In [1]:
from google.colab import drive
drive.mount(
    '/content/drive',
    force_remount=True
)

Mounted at /content/drive


In [2]:
import pandas as pd


In [3]:
ground_truth = pd.read_csv('/content/drive/MyDrive/senior_thesis/comparison_essay_grading/paired_essays/overall_paired_essays.csv')
ground_truth = ground_truth.sample(n=100, random_state=42)
ground_truth = ground_truth.set_index(['id_1', 'id_2'])
ground_truth.head(10)

,,text_1,text_2,1_better,prompt
id_1,id_2,,,,
A828480AA508,AA8DA5629CBC,"Successes comes in so many diffrent whays,but ...",Why is it good to have a good attitude even if...,1,Benefits of a good attitude
65B7FDE783F2,1732E7BCD28D,"I am student for Generic_School,I have work af...",The worker in the company need to conserve the...,0,Four-day work week
7DC02C918305,98BD2D5840A1,It has been said that a positive is the key to...,Having a positive attitude can created more op...,0,Positive attitudes
6060D28C05B6,5AC16E188F1B,Some schools in United States ofter classes fr...,Today technology is arriving into the schools....,0,Distance learning
3F3DC123EDF6,CBFD5CD8BCD9,"Do curfews keep teenagers out of trouble, or d...","Yes, I believe curfew can keep citizens out of...",1,Curfews for teenagers
A8E686531564,9C9F45A47A92,Some schools use cell phones in classrooms for...,With can use cell phone in the classroom becau...,1,Cell phones in classrooms
E3668C4C4C0C,C2AECED57C95,"Kids these days want to have fun, hang out wit...","DEAR, Mr. Middle School Principal\r\n\r\nI Agr...",1,Community service
04776677F17B,D49CB44F4364,"Graduating, everybody wants to do it right? I ...",Some school shouldn't offer programs that allo...,0,Three-year high school program
93385DED39E7,A202FBAF0CCD,Some schools offer distance learning as an opt...,Some students should be offered distance learn...,1,Distance learning


In [4]:
import pandas as pd
import os

folder_path = '/content/drive/MyDrive/senior_thesis/comparison_essay_grading/outputs'
dfs = {}
for filename in os.listdir(folder_path):
  if filename.endswith('.csv'):
    print(f"Processing: {filename}")

    file_path = os.path.join(folder_path, filename)
    df = pd.read_csv(file_path)
    filename = filename[:-4]
    df = df.set_index(['id_1', 'id_2'])
    df['pred'] = df['pred'].astype(str).str.strip()
    #A(1) is better = 1
    df['pred'] = df['pred'].map({'A': 1, 'B': 0})
    dfs[filename] = df


Processing: qwen-2.5-7b-it.csv
Processing: qwen-2.5-0.5b-it.csv
Processing: qwen-2.5-7b-it-1m.csv
Processing: qwen-2.5-1.5b-it.csv
Processing: qwen-2.5-3b-it.csv
Processing: llama-3.2-3b-it.csv
Processing: gemma-2b.csv
Processing: gemma-7b.csv
Processing: gemma-2-2b_raw.csv
Processing: llama-3-8b-it.csv
Processing: llama-3.2-1b-it.csv
Processing: llama-3.2-3b.csv
Processing: gemma-1.1-2b-it.csv
Processing: gemma-2-2b-it.csv
Processing: phi-3-medium-4k-instruct.csv
Processing: phi-3-mini-4k-instruct.csv
Processing: phi-3.5-mini-instruct.csv
Processing: mistral-7B-instruct-v0.1.csv
Processing: mistral-8B-instruct-2410.csv
Processing: mistral-nemo-instruct-2407.csv
Processing: mistral-7B-instruct-v0.3.csv
Processing: olmo-7b-it.csv
Processing: olmoe-1b-7b-0924-it.csv
Processing: gpt2-small.csv
Processing: gpt2-xl.csv
Processing: gpt2-large.csv
Processing: gpt2-medium.csv
Processing: phi-4-mini-instruct.csv
Processing: flan-t5-base.csv
Processing: flan-t5-xl.csv
Processing: flan-t5-small.c

In [6]:
dfs['flan-t5-large'].head(5)

,,prompt,pred
id_1,id_2,,
A828480AA508,AA8DA5629CBC,Benefits of a good attitude,1
65B7FDE783F2,1732E7BCD28D,Four-day work week,0
7DC02C918305,98BD2D5840A1,Positive attitudes,0
6060D28C05B6,5AC16E188F1B,Distance learning,0
3F3DC123EDF6,CBFD5CD8BCD9,Curfews for teenagers,0


In [7]:
for name, df_old in dfs.items():
    df_old = df_old.join(ground_truth[['1_better']], how='left')  # Join on index
    df_old['correct'] = (df_old['pred'] == df_old['1_better']).astype(int) # Compare predictions
    dfs[name] = df_old.drop(columns=['pred', '1_better'])  # Clean up by removing extra column

In [9]:
dfs['flan-t5-large'].head(5)

,,prompt,correct
id_1,id_2,,
A828480AA508,AA8DA5629CBC,Benefits of a good attitude,1
65B7FDE783F2,1732E7BCD28D,Four-day work week,1
7DC02C918305,98BD2D5840A1,Positive attitudes,1
6060D28C05B6,5AC16E188F1B,Distance learning,1
3F3DC123EDF6,CBFD5CD8BCD9,Curfews for teenagers,0


In [11]:
for name, df in dfs.items():
    df.to_csv(f'/content/drive/MyDrive/senior_thesis/comparison_essay_grading/final_outputs/{name}.csv', index=True)

In [ ]:
dfs.items()